<a href="https://colab.research.google.com/github/ifankang/ExcelCleanerPolar/blob/main/excel_cleaning_polar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Clean Excel Files
import os
import pandas as pd
import time

def find_header_row(df_preview, min_non_nulls=3):
    for idx, row in df_preview.iterrows():
        if row.notna().sum() >= min_non_nulls:
            return idx
    return None

def _clean_column_names(columns):
    cleaned = pd.Series(columns).astype(str).str.strip().str.lower()
    cleaned = cleaned.str.replace(r"[^\w]+", "_", regex=True).str.strip("_")
    cleaned = cleaned.str.replace(r"_{2,}", "_", regex=True)

    for dup in cleaned[cleaned.duplicated()].unique():
        idxs = cleaned[cleaned == dup].index
        cleaned.loc[idxs] = [f"{dup}_{i}" if i else dup for i in range(len(idxs))]

    return cleaned

def cleaning_df(df: pd.DataFrame) -> pd.DataFrame | None:
    """
    Cleans a pandas DataFrame that originated from an Excel file.

    Args:
        df: The input pandas DataFrame.

    Returns:
        A cleaned pandas DataFrame or None if cleaning fails.
    """
    print("Starting DataFrame cleaning...")


    try:
        # Step 1: Find header row (using the find_header_row logic adapted for DataFrame)
        # We can't use a file path anymore, so we need a way to detect the header row
        # within the DataFrame itself. We'll look at the first few rows.
        df_preview = df.head(20).copy() # Look at the first 20 rows for header detection

        header_row_idx = None
        # Assuming the header is a row where a reasonable number of columns are not null
        # This is a heuristic and might need adjustment based on the specific data structure
        for idx in range(len(df_preview)):
             # Check if at least 3 non-null values exist in this row
            if df_preview.iloc[idx].notna().sum() >= 3:
                header_row_idx = idx
                break


        if header_row_idx is None:
            print(f"⚠️ Could not find valid header in the provided DataFrame.")
            return None

        # # Step 2: Extract data from the detected header row onwards
        # # The actual headers are in the detected header row (header_row_idx)
        headers = df.iloc[header_row_idx].tolist()
        # # The data starts from the row *after* the header row
        df_data = df.iloc[header_row_idx+1:].copy()

        df_data.columns = headers
        df_data = df_data.dropna(how="all", axis=1)     # Drop fully empty columns
        # Drop mostly empty rows (rows with less than 50% non-null values)
        df_data = df_data.dropna(thresh=int(df_data.shape[1] * 0.5))
        df_data = df_data.ffill()                       # Forward-fill missing values


        # Step 3: Clean column names to remove duplicates and unsafe characters
        df_data.columns = _clean_column_names(df_data.columns)


        print("✅ DataFrame cleaning complete.")
        return df_data.reset_index(drop=True)

    except Exception as e:

        print(f"❌ Error cleaning DataFrame: {e}")
        return None

# Ensure the cleaned_folder exists
start_time = time.time()
cleaned_folder = "cleaned"
if not os.path.exists(cleaned_folder):
    os.makedirs(cleaned_folder)

# Iterate through files in the current directory
for filename in os.listdir("."):
    if filename.endswith(".xlsx") or filename.endswith(".xls"):
        print(f"Processing {filename}...")
        try:
            # Read the excel file
            df = pd.read_excel(filename, engine='calamine')

            # Clean the dataframe
            cleaned_df = cleaning_df(df.copy())

            if cleaned_df is not None:
                # Define the output filename
                output_filename = os.path.join(cleaned_folder, f"cleaned_{filename}")
                end_time = time.time()
                elapsed_time = end_time - start_time
                print(f"Time elapsed: {elapsed_time:.4f} seconds")
                start_time = time.time()
                # Save the cleaned dataframe to a new excel file
                cleaned_df.to_excel(output_filename, index=False, engine='xlsxwriter')
                end_time = time.time()
                elapsed_time = end_time - start_time
                print(f"Time exporting to excel: {elapsed_time:.4f} seconds")
                print(f"Saved cleaned data to {output_filename}")
            else:
                end_time = time.time()
                elapsed_time = end_time - start_time
                print(f"Time elapsed: {elapsed_time:.4f} seconds")
                print(f"Skipping saving for {filename} due to cleaning failure.")

        except Exception as e:
            end_time = time.time()
            elapsed_time = end_time - start_time
            print(f"Time elapsed before error: {elapsed_time:.4f} seconds")
            print(f"Error processing {filename}: {e}")

In [ ]:
cleaned_df.head(20)

In [ ]:
# @title Clean Excel (Faster)
%pip install -q fastexcel xlsxwriter polars
import os
import polars as pl
import time
import re
import logging
logging.getLogger('fastexcel.types.dtype').disabled = True

def find_header_row_pl(df_preview: pl.DataFrame, min_non_nulls=3) -> int | None:
    """
    Finds the index of the potential header row in a Polars DataFrame preview.
    A row is considered a header row if it has at least `min_non_nulls` non-null values.
    """
    # print("Debug: Finding header row...")
    for idx in range(df_preview.height):
        row = df_preview.row(idx, named=False) # Get row as a tuple
        # Count non-nulls in the tuple
        non_null_count = sum(1 for x in row if x is not None and x != '') # Add check for empty strings
        # print(f"Debug: Row {idx}: Non-null count = {non_null_count}")
        if non_null_count >= min_non_nulls:
            # print(f"Debug: Found potential header row at index {idx}")
            return idx
    # print("Debug: No valid header row found.")
    return None

def _clean_column_names_pl(columns: list[str]) -> list[str]:
    """
    Cleans a list of column names for use in Polars DataFrame.
    Replaces non-alphanumeric characters with underscores, removes leading/trailing underscores,
    collapses multiple underscores, and handles duplicates by appending a number.
    """
    # print("Debug: Cleaning column names...")
    cleaned = [col.strip().lower() if col is not None else "" for col in columns] # Handle None values
    cleaned = [re.sub(r"[^\w]+", "_", col) for col in cleaned]
    cleaned = [col.strip("_") for col in cleaned]
    cleaned = [re.sub(r"_{2,}", "_", col) for col in cleaned]

    counts = {}
    cleaned_final = []
    for col in cleaned:
        if col in counts:
            counts[col] += 1
            cleaned_final.append(f"{col}_{counts[col]}")
            print(f"Debug: Renaming duplicate column '{col}' to '{col}_{counts[col]}'")
        else:
            counts[col] = 0
            cleaned_final.append(col)
            # print(f"Debug: Keeping column name '{col}'")

    # print("Debug: Column names cleaned.")
    return cleaned_final


def cleaning_df_polars(df: pl.DataFrame) -> pl.DataFrame | None:
    """
    Cleans a Polars DataFrame that originated from an Excel file.

    Args:
        df: The input Polars DataFrame.

    Returns:
        A cleaned Polars DataFrame or None if cleaning fails.
    """
    print("Starting Polars DataFrame cleaning...")


    try:
        # Step 1: Find header row
        df_preview = df.head(20) # Look at the first 20 rows for header detection
        header_row_idx = find_header_row_pl(df_preview)

        if header_row_idx is None:
            print(f"⚠️ Could not find valid header in the provided DataFrame.")
            return None



        # Step 2: Extract data from the detected header row onwards
        # The actual headers are in the detected header row (header_row_idx)
        headers = df.row(header_row_idx, named=False)
        cleaned_columns = _clean_column_names_pl(headers)
        # print(f"Debug: Cleaned Headers: {cleaned_columns}")


        # The data starts from the row *after* the header row
        # Slice the DataFrame first
        df_data = df.slice(offset=header_row_idx + 1, length=df.height - (header_row_idx + 1))



        # Now rename the sliced DataFrame using the cleaned headers
        # Create a dictionary mapping the original column names of df_data to the cleaned column names
        # The original column names are still the __UNNAMED ones from pl.read_excel
        rename_dict = {old_col: new_col for old_col, new_col in zip(df_data.columns, cleaned_columns)}
        df_data = df_data.rename(rename_dict)
        # print(f"Debug: Rename dictionary: {rename_dict}")
        # print(f"Debug: DataFrame after renaming columns and slicing data. Shape: {df_data.shape}")
        # print(f"Debug: Columns after renaming: {df_data.columns}")

        # Step 4: Clean data - Drop fully empty columns, drop mostly empty rows, ffill
        # Drop fully empty columns (check if all values are null or empty string)
        cols_to_drop = [col for col in df_data.columns if df_data[col].is_null().all() or (df_data[col] == '').all()]
        if cols_to_drop:
            # print(f"Debug: Dropping fully empty columns: {cols_to_drop}")
            df_data = df_data.drop(cols_to_drop)
            # print(f"Debug: DataFrame shape after dropping empty columns: {df_data.shape}")

        # Drop mostly empty rows (rows with less than 25% non-null values)
        # Calculate minimum non-null values required per row
        min_non_null_per_row = int(df_data.shape[1] * 0.25)
        print(f"Debug: Dropping rows with less than {min_non_null_per_row} non-null values.")

        # Calculate row-wise non-null counts, excluding empty strings
        # Use a list comprehension to create expressions for each column
        non_null_exprs = [
            pl.when(pl.col(c).is_not_null() & (pl.col(c).cast(pl.Utf8) != '')).then(1).otherwise(0)
            for c in df_data.columns
        ]

        # Sum the non-null counts across columns for each row and filter
        df_data = df_data.filter(
            pl.sum_horizontal(non_null_exprs) >= min_non_null_per_row
        )


        # Remove completely empty rows
        # df_data = df_data.filter(
        #   pl.concat_list([pl.col(col).is_not_null() & (pl.col(col) != "") for col in df.columns])
        # ).any(axis=1)

        # Drop mostly empty rows (rows with less than 50% non-null values
        # # Calculate minimum non-null values required per row
        # min_non_null_per_row = int(df_data.shape[1] * 0.5)
        # print(f"Debug: Dropping rows with less than {min_non_null_per_row} non-null values.")

        # # Calculate row-wise non-null counts, excluding empty strings
        # row_non_null_counts = df_data.select([
        #     (pl.when(pl.col(c).is_not_null() & (pl.col(c).cast(pl.Utf8) != '')).then(1).otherwise(0)).sum().alias(c) for c in df_data.columns
        # ]).sum().to_series()  # Convert to Series

        # # Print non-null counts for debugging
        # print("Debug: Non-null counts per row:")
        # print(row_non_null_counts)

        # # Filter rows based on the calculated non-null counts
        # df_data = df_data.filter(row_non_null_counts >= min_non_null_per_row)
        # print(f"Debug: DataFrame shape after dropping mostly empty rows: {df_data.shape}")


        # Forward-fill missing values
        # Polars ffill works column-wise
        df_data = df_data.fill_null(strategy="forward")
        # Also consider filling empty strings with the previous non-empty value if needed
        # This is more complex and depends on the data type and desired behavior.
        # For simplicity, fill nulls only.
        return df_data

    except Exception as e:
        print(f"❌ Error cleaning Polars DataFrame: {e}")
        return None

def polars_dataframe_dtype_validate(df: pl.DataFrame) -> pl.DataFrame:
    """
    Infers the data type for each column in a Polars DataFrame based on its content
    and casts the column to the inferred type.

    Args:
        df: The input Polars DataFrame.

    Returns:
        A Polars DataFrame with column types validated and casted based on content.
    """
    print("Starting Polars DataFrame dtype validation...")
    new_df = df.clone() # Create a copy to avoid modifying the original DataFrame in place

    for col in new_df.columns:
        # print(f"Debug: Processing column '{col}'...")
        # Skip if the column is entirely null
        if new_df[col].is_null().all():
            # print(f"Debug: Column '{col}' is all null, keeping as Utf8.")
            continue # Keep the default Utf8 type for all-null columns

        # Get the first non-null, non-empty string value to infer type
        first_value = None
        for value in new_df[col].to_list():
            if value is not None and value != '' and (isinstance(value, str) and value.strip() != '' or not isinstance(value, str)):
                 first_value = value
                 break

        inferred_type = pl.Utf8 # Default type

        if first_value is not None:
             # Try to infer a more specific type
             if isinstance(first_value, (int, float)):
                 inferred_type = pl.Float64
             elif isinstance(first_value, str):
                 try:
                     # Attempt to parse as date
                     # Use an explicit format string based on the observed data
                     pl.from_epoch(new_df[col].str.strptime(pl.Datetime, "%Y-%m-%d %H:%M:%S"), time_unit="ms") # Check if can be parsed as Datetime first
                     inferred_type = pl.Date # If successful, we can cast to Date
                 except Exception: # Catch any exception during parsing
                     try:
                         # Attempt to parse as a number (float)
                         float(first_value)
                         inferred_type = pl.Float64
                     except (ValueError, TypeError):
                          # If neither date nor number, keep as string
                         inferred_type = pl.Utf8
             elif isinstance(first_value, pd.Timestamp):
                 inferred_type = pl.Date

        # print(f"Debug: Inferred type for column '{col}': {inferred_type}")

        # Attempt to cast the column to the inferred type
        try:
            if inferred_type == pl.Date:
                # Cast to Utf8 first if not already, then attempt parsing to Datetime
                if new_df[col].dtype != pl.Utf8:
                     new_df = new_df.with_columns(pl.col(col).cast(pl.Utf8).alias(col))

                # Parse as Datetime and then cast to Date
                new_df = new_df.with_columns(pl.col(col).str.strptime(pl.Datetime, "%Y-%m-%d %H:%M:%S", strict=False).cast(pl.Date).alias(col))
                # print(f"Debug: Column '{col}' successfully casted to Date.")
            elif inferred_type == pl.Float64:
                 # Attempt to cast to Float64, coercing errors to null
                 new_df = new_df.with_columns(pl.col(col).cast(pl.Float64, strict=False).alias(col))
                #  print(f"Debug: Column '{col}' successfully casted to Float64.")
            elif new_df[col].dtype != inferred_type: # Avoid unnecessary recasting
                 new_df = new_df.with_columns(pl.col(col).cast(inferred_type).alias(col))
                #  print(f"Debug: Column '{col}' successfully casted to {inferred_type}.")
            # else:
            #     print(f"Debug: Column '{col}' already has the inferred type {inferred_type}, no casting needed.")


        except Exception as e:
            # print(f"Debug: Could not cast column '{col}' to {inferred_type}: {e}. Keeping original dtype.")
            # If casting fails, keep original dtype or Utf8 if that was the original
            pass # Do nothing, keep the current dtype


    print("✅ Polars DataFrame dtype validation complete.")
    return new_df



# Ensure the cleaned_folder exists
cleaned_folder = "cleaned_polars"
if not os.path.exists(cleaned_folder):
    os.makedirs(cleaned_folder)

# Iterate through files in the current directory
start_time = time.time()
for filename in os.listdir("."):
    if filename.endswith(".xlsx") or filename.endswith(".xls"):
        print(f"\nProcessing {filename} with Polars...")
        try:
            # Read the excel file using Polars with FastExcel warper
            df = pl.read_excel(filename)
            # Clean the dataframe
            cleaned_df_pl = cleaning_df_polars(df)

            if cleaned_df_pl is not None:
                # Define the output filename
                output_filename = os.path.join(cleaned_folder, f"cleaned_polars_{filename}")

                end_time = time.time()
                elapsed_time = end_time - start_time
                print("✅ Polars DataFrame cleaning complete.")
                print(f"Time cleaning: {elapsed_time:.4f} seconds")
                start_time = time.time()
                #validate dtype of Polars DF because the FastExcel return all column as String
                cleaned_df_pl=polars_dataframe_dtype_validate(cleaned_df_pl)
                #write to excel
                print("Saving...")
                cleaned_df_pl.write_excel(output_filename)
                end_time = time.time()
                elapsed_time = end_time - start_time
                print("✅ Polars DataFrame cleaning complete.")
                print(f"Time exporting to excel: {elapsed_time:.4f} seconds")
                print(f"Saved cleaned Polars data to {output_filename}")
            else:
                end_time = time.time()
                elapsed_time = end_time - start_time
                print("✅ Polars DataFrame cleaning complete.")
                print(f"Time elapsed: {elapsed_time:.4f} seconds")
                print(f"Skipping saving for {filename} due to cleaning failure.")

        except Exception as e:
            end_time = time.time()
            elapsed_time = end_time - start_time
            print("✅ Polars DataFrame cleaning complete.")
            print(f"Time elapsed: {elapsed_time:.4f} seconds")
            print(f"Error processing {filename} with Polars: {e}")

In [ ]:
cleaned_df_pl.head(20)


In [ ]:
# @title Excel Data Cleaner (Powered by Polars)

# @markdown This notebook helps you clean messy Excel files by automatically detecting headers, cleaning column names,
# @markdown dropping empty rows and columns, and inferring data types.
# @markdown It processes all `.xlsx` and `.xls` files in the current Google Colab directory.

# @markdown ---

# @markdown ### How to Use:
# @markdown 1. **Upload your Excel files:** Drag and drop your `.xlsx` or `.xls` files directly into the Google Colab file browser (left sidebar -> folder icon).
# @markdown 2. **Run the code cell:** Click the "Play" button next to this code cell, or press `Ctrl + Enter` (Windows/Linux) / `Cmd + Enter` (Mac).
# @markdown 3. **Monitor the output:** The cell will display progress messages and indicate when cleaning is complete.
# @markdown 4. **Download cleaned files:** A new folder named `cleaned_polars` will be created in your Colab environment,
# @markdown    containing the cleaned versions of your Excel files (e.g., `cleaned_polars_yourfile.xlsx`).
# @markdown    You can download them from the file browser.

# @markdown ---

# @markdown ### Configuration:
# @markdown Adjust these parameters if the default behavior isn't suitable for your data.

min_header_non_nulls=3 # @param {type:"integer"}
# @markdown Minimum number of non-null values required in a row to be considered a potential header row.
# @markdown Increase this if your header rows are sparse, decrease if they are very dense and some data rows might be mistaken for headers.

min_row_non_null_percentage=0.25 # @param {type:"number"}
# @markdown Minimum percentage (0.0 to 1.0) of non-null values a row must have to avoid being dropped.
# @markdown Rows with fewer non-nulls than this percentage will be considered "mostly empty" and removed.


# @markdown ---

# @markdown ### Important Considerations:
# @markdown * **File Location:** Ensure your Excel files are in the *same directory* as this notebook in Google Colab.
# @markdown * **Large Files:** For very large Excel files, the processing time might be significant.
# @markdown * **Date Format:** The date inference currently attempts to parse dates in `YYYY-MM-DD HH:MM:SS` format. If your dates are in a different format, they might remain as strings or be incorrectly parsed. Future improvements could include more robust date parsing.
# @markdown * **Error Handling:** While some error handling is in place, highly malformed Excel files might still cause issues.
# @markdown * **Output Folder:** Cleaned files are saved in a subfolder named `cleaned_polars`. If you run this multiple times, existing files with the same name in this folder will be overwritten.


%pip install -q fastexcel xlsxwriter polars
import os
import polars as pl
import time
import re
import logging
logging.getLogger('fastexcel.types.dtype').disabled = True

def find_header_row_pl(df_preview: pl.DataFrame, min_non_nulls: int) -> int | None:
    """
    Finds the index of the potential header row in a Polars DataFrame preview.
    A row is considered a header row if it has at least `min_non_nulls` non-null values.
    """
    for idx in range(df_preview.height):
        row = df_preview.row(idx, named=False)
        non_null_count = sum(1 for x in row if x is not None and str(x).strip() != '')
        if non_null_count >= min_non_nulls:
            return idx
    return None

def _clean_column_names_pl(columns: list[str]) -> list[str]:
    """
    Cleans a list of column names for use in Polars DataFrame.
    Replaces non-alphanumeric characters with underscores, removes leading/trailing underscores,
    collapses multiple underscores, and handles duplicates by appending a number.
    """
    cleaned = [col.strip().lower() if col is not None else "" for col in columns]
    cleaned = [re.sub(r"[^\w]+", "_", col) for col in cleaned]
    cleaned = [col.strip("_") for col in cleaned]
    cleaned = [re.sub(r"_{2,}", "_", col) for col in cleaned]

    counts = {}
    cleaned_final = []
    for col in cleaned:
        if col in counts:
            counts[col] += 1
            cleaned_final.append(f"{col}_{counts[col]}")
            print(f"  ↪ Renaming duplicate column '{col}' to '{col}_{counts[col]}'")
        else:
            counts[col] = 0
            cleaned_final.append(col)
    return cleaned_final


def cleaning_df_polars(df: pl.DataFrame, min_header_non_nulls: int, min_row_non_null_percentage: float) -> pl.DataFrame | None:
    """
    Cleans a Polars DataFrame that originated from an Excel file.

    Args:
        df: The input Polars DataFrame.
        min_header_non_nulls: Minimum non-nulls for header detection.
        min_row_non_null_percentage: Minimum percentage of non-nulls for a row to be kept.

    Returns:
        A cleaned Polars DataFrame or None if cleaning fails.
    """
    print("  Starting Polars DataFrame cleaning...")

    try:
        # Step 1: Find header row
        df_preview = df.head(20) # Look at the first 20 rows for header detection
        header_row_idx = find_header_row_pl(df_preview, min_header_non_nulls)

        if header_row_idx is None:
            print(f"  ⚠️ Could not find a valid header in the provided DataFrame. Skipping cleaning.")
            return None

        print(f"  ✅ Detected header row at index: {header_row_idx}")

        # Step 2: Extract data from the detected header row onwards
        headers = df.row(header_row_idx, named=False)
        cleaned_columns = _clean_column_names_pl(headers)

        df_data = df.slice(offset=header_row_idx + 1, length=df.height - (header_row_idx + 1))

        rename_dict = {old_col: new_col for old_col, new_col in zip(df_data.columns, cleaned_columns)}
        df_data = df_data.rename(rename_dict)
        print(f"  ✅ Columns renamed and data extracted.")

        # Step 3: Clean data - Drop fully empty columns, drop mostly empty rows, ffill
        cols_to_drop = [col for col in df_data.columns if df_data[col].is_null().all() or (df_data[col] == '').all()]
        if cols_to_drop:
            print(f"  🗑️ Dropping fully empty columns: {', '.join(cols_to_drop)}")
            df_data = df_data.drop(cols_to_drop)

        # Drop mostly empty rows (rows with less than min_row_non_null_percentage non-null values)
        min_non_null_per_row = int(df_data.shape[1] * min_row_non_null_percentage)
        print(f"  🗑️ Dropping rows with less than {min_non_null_per_row} non-null values (based on {min_row_non_null_percentage*100}% threshold).")

        non_null_exprs = [
            pl.when(pl.col(c).is_not_null() & (pl.col(c).cast(pl.Utf8) != '')).then(1).otherwise(0)
            for c in df_data.columns
        ]

        df_data = df_data.filter(
            pl.sum_horizontal(non_null_exprs) >= min_non_null_per_row
        )
        print(f"  ✅ Remaining rows after dropping mostly empty: {df_data.shape[0]}")

        # Forward-fill missing values
        df_data = df_data.fill_null(strategy="forward")
        print("  ✅ Forward-filled missing values.")

        return df_data

    except Exception as e:
        print(f"  ❌ Error cleaning Polars DataFrame: {e}")
        return None

def polars_dataframe_dtype_validate(df: pl.DataFrame) -> pl.DataFrame:
    """
    Infers the data type for each column in a Polars DataFrame based on its content
    and casts the column to the inferred type.

    Args:
        df: The input Polars DataFrame.

    Returns:
        A Polars DataFrame with column types validated and casted based on content.
    """
    print("Starting Polars DataFrame dtype validation...")
    new_df = df.clone() # Create a copy to avoid modifying the original DataFrame in place

    for col in new_df.columns:
        # print(f"Debug: Processing column '{col}'...")
        # Skip if the column is entirely null
        if new_df[col].is_null().all():
            # print(f"Debug: Column '{col}' is all null, keeping as Utf8.")
            continue # Keep the default Utf8 type for all-null columns

        # Get the first non-null, non-empty string value to infer type
        first_value = None
        for value in new_df[col].to_list():
            if value is not None and value != '' and (isinstance(value, str) and value.strip() != '' or not isinstance(value, str)):
                 first_value = value
                 break

        inferred_type = pl.Utf8 # Default type

        if first_value is not None:
             # Try to infer a more specific type
             if isinstance(first_value, (int, float)):
                 inferred_type = pl.Float64
             elif isinstance(first_value, str):
                 try:
                     # Attempt to parse as date
                     # Use an explicit format string based on the observed data
                     pl.from_epoch(new_df[col].str.strptime(pl.Datetime, "%Y-%m-%d %H:%M:%S"), time_unit="ms") # Check if can be parsed as Datetime first
                     inferred_type = pl.Date # If successful, we can cast to Date
                 except Exception: # Catch any exception during parsing
                     try:
                         # Attempt to parse as a number (float)
                         float(first_value)
                         inferred_type = pl.Float64
                     except (ValueError, TypeError):
                          # If neither date nor number, keep as string
                         inferred_type = pl.Utf8
             elif isinstance(first_value, pd.Timestamp):
                 inferred_type = pl.Date

        # print(f"Debug: Inferred type for column '{col}': {inferred_type}")

        # Attempt to cast the column to the inferred type
        try:
            if inferred_type == pl.Date:
                # Cast to Utf8 first if not already, then attempt parsing to Datetime
                if new_df[col].dtype != pl.Utf8:
                     new_df = new_df.with_columns(pl.col(col).cast(pl.Utf8).alias(col))

                # Parse as Datetime and then cast to Date
                new_df = new_df.with_columns(pl.col(col).str.strptime(pl.Datetime, "%Y-%m-%d %H:%M:%S", strict=False).cast(pl.Date).alias(col))
                # print(f"Debug: Column '{col}' successfully casted to Date.")
            elif inferred_type == pl.Float64:
                 # Attempt to cast to Float64, coercing errors to null
                 new_df = new_df.with_columns(pl.col(col).cast(pl.Float64, strict=False).alias(col))
                #  print(f"Debug: Column '{col}' successfully casted to Float64.")
            elif new_df[col].dtype != inferred_type: # Avoid unnecessary recasting
                 new_df = new_df.with_columns(pl.col(col).cast(inferred_type).alias(col))
                #  print(f"Debug: Column '{col}' successfully casted to {inferred_type}.")
            # else:
            #     print(f"Debug: Column '{col}' already has the inferred type {inferred_type}, no casting needed.")


        except Exception as e:
            # print(f"Debug: Could not cast column '{col}' to {inferred_type}: {e}. Keeping original dtype.")
            # If casting fails, keep original dtype or Utf8 if that was the original
            pass # Do nothing, keep the current dtype


    print("✅ Polars DataFrame dtype validation complete.")
    return new_df


# Ensure the cleaned_folder exists
cleaned_folder = "cleaned_polars"
if not os.path.exists(cleaned_folder):
    os.makedirs(cleaned_folder)
    print(f"Created output folder: {cleaned_folder}")

print("\n--- Starting Excel Cleaning Process ---")
total_start_time = time.time()
processed_files_count = 0

# Iterate through files in the current directory
for filename in os.listdir("."):
    if filename.endswith(".xlsx") or filename.endswith(".xls"):
        processed_files_count += 1
        print(f"\n✨ Processing file: {filename}...")
        file_start_time = time.time()
        try:
            # Read the excel file using Polars with FastExcel wrapper
            # Use `read_excel_fastexcel` to ensure the wrapper is used for performance
            df = pl.read_excel(filename) # Specify engine for clarity

            # Clean the dataframe
            cleaned_df_pl = cleaning_df_polars(df, min_header_non_nulls, min_row_non_null_percentage)

            if cleaned_df_pl is not None:
                # Define the output filename
                output_filename = os.path.join(cleaned_folder, f"cleaned_polars_{filename}")

                # Validate and cast dtypes
                cleaned_df_pl = polars_dataframe_dtype_validate(cleaned_df_pl)

                # Write to excel
                print("  💾 Saving cleaned DataFrame...")
                cleaned_df_pl.write_excel(output_filename) # xlsxwriter is needed for writing

                file_end_time = time.time()
                elapsed_file_time = file_end_time - file_start_time
                print(f"  ✅ Successfully processed and saved to {output_filename} in {elapsed_file_time:.2f} seconds.")
            else:
                file_end_time = time.time()
                elapsed_file_time = file_end_time - file_start_time
                print(f"  ❌ Skipping saving for {filename} due to cleaning failure (took {elapsed_file_time:.2f} seconds).")

        except Exception as e:
            file_end_time = time.time()
            elapsed_file_time = file_end_time - file_start_time
            print(f"  ❌ Error processing {filename}: {e} (took {elapsed_file_time:.2f} seconds).")

if processed_files_count == 0:
    print("\nNo Excel files (.xlsx or .xls) found in the current directory to process.")

total_end_time = time.time()
total_elapsed_time = total_end_time - total_start_time
print(f"\n--- Excel Cleaning Process Complete! Total time: {total_elapsed_time:.2f} seconds ---")


--- Starting Excel Cleaning Process ---

✨ Processing file: rincian_faktur_penjualan_ptgoeboekinfota_250428102742.xlsx...
  Starting Polars DataFrame cleaning...
  ✅ Detected header row at index: 2
  ↪ Renaming duplicate column 'diskon' to 'diskon_1'
  ↪ Renaming duplicate column 'diskon' to 'diskon_2'
  ↪ Renaming duplicate column 'diskon' to 'diskon_3'
  ↪ Renaming duplicate column '' to '_1'
  ✅ Columns renamed and data extracted.
  🗑️ Dropping rows with less than 6 non-null values (based on 25.0% threshold).
  ✅ Remaining rows after dropping mostly empty: 130278
  ✅ Forward-filled missing values.
Starting Polars DataFrame dtype validation...
✅ Polars DataFrame dtype validation complete.
  💾 Saving cleaned DataFrame...
  ✅ Successfully processed and saved to cleaned_polars/cleaned_polars_rincian_faktur_penjualan_ptgoeboekinfota_250428102742.xlsx in 56.95 seconds.

--- Excel Cleaning Process Complete! Total time: 56.95 seconds ---


In [ ]:
#@title Cleaned excel preview (first 10 rows)
cleaned_df_pl.head(10)

,tanggal,pelanggan,nomor,kategori,kode,nama_barang,gudang,kuantitas,harga,diskon,diskon_1,diskon_2,diskon_3,total_penjualan_setelah_discount_inc_ppn,total_sebelum_diskon_per_faktur,bpp_inc_ppn,bpp_qty_inc_ppn,laba_satuan_inc_ppn,laba_kotor_total,no_order,sales,keterangan,total_termasuk_pajak_faktur_penjualan,kena_pajak_faktur_penjualan,tipe_pajak_ppn_barang_jasa,_1
str,date,str,str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str,str,str,str
null,2024-02-02,"""YOGYA KEPATIHAN""","""GIT.2024.02.00014""","""OLIKE ACC""","""LDU SF4 BLUE""","""LDU OLIKE BEATZ WIRELESS PORTA…","""YOGYA KEPATIHAN""",1.0,143200.0,17900.0,12.5,0.0,null,125300.13,125300.0,83399.9943,83399.9943,41900.1357,41900.1357,null,"""CC-CHRISTIAN CHANDRA""",null,"""Ya""","""Ya""","""Pajak Pertambahan Nilai""",null
null,2024-02-03,"""TIKTOK""","""OL.2024.02.00192""","""MAKUKU""","""MAKUKU ADDCPM30""","""MAKUKU DRY CARE PANTS M30""","""ONLINE""",1.0,44299.0,0.0,12.5,0.0,null,44300.1,44299.0,43919.925,43919.925,380.175,380.175,5.7847e17,"""TK2FN-FEIZAL""",null,"""Ya""","""Ya""","""Pajak Pertambahan Nilai""",null
null,2024-02-02,"""YOGYA SUBANG""","""GIT.2024.02.00008""","""OLIKE ACC""","""T112 WHITE""","""OLIKE TRUE WIRELESS EARPHONES …","""YOGYA SUBANG""",1.0,144560.0,38970.0,12.5,0.0,null,126490.27,126490.0007,104999.99934,104999.99934,21490.270659,21490.27066,5.7847e17,"""CC-CHRISTIAN CHANDRA""","""01 februari 2024 GIT ACC PROMO""","""Ya""","""Ya""","""Pajak Pertambahan Nilai""",null
null,2024-02-02,"""YOGYA SUBANG""","""GIT.2024.02.00008""","""OLIKE ACC""","""P101S DARK BLUE""","""OLIKE DIGITAL DISPL POWERBANK1…","""YOGYA SUBANG""",1.0,167200.0,38970.0,12.5,0.0,null,146300.0,146300.03827,136999.99953,136999.99953,9300.00047,9300.00047,5.7847e17,"""CC-CHRISTIAN CHANDRA""","""01 februari 2024 GIT ACC PROMO""","""Ya""","""Ya""","""Pajak Pertambahan Nilai""",null
null,2024-02-03,"""TIKTOK""","""OL.2024.02.00235""","""MAKUKU""","""MAKUKU ADDCP L28""","""MAKUKU DRY CARE PANTS L28""","""ONLINE""",1.0,44299.0,0.0,12.5,0.0,null,44300.1,44299.0,43919.925,43919.925,380.175,380.175,5.7848e17,"""TK2FN-FEIZAL""","""01 februari 2024 GIT ACC PROMO""","""Ya""","""Ya""","""Pajak Pertambahan Nilai""",null
null,2024-02-02,"""SHOPEE OL""","""OL.2024.02.00066""","""MAKUKU""","""MAKUKU TSM ADSPXL32N""","""MAKUKU TSM SLIM PANTS XL32 NEW""","""ONLINE""",5.0,102600.0,0.0,12.5,0.0,null,513000.329999,513000.0,94856.16,474280.8,7743.905999,38719.529999,null,"""SP1FN-FEIZAL""","""INSTAN""","""Ya""","""Ya""","""Pajak Pertambahan Nilai""",null
null,2024-02-02,"""SHOPEE OL""","""OL.2024.02.00066""","""MAKUKU""","""MAKUKU AD SCP XL46""","""MAKUKU SLIM CARE PANTS XL46""","""ONLINE""",5.0,120999.0,0.0,12.5,0.0,null,604995.000001,604995.0,108083.808,540419.04,12915.192,64575.960001,null,"""SP1FN-FEIZAL""","""INSTAN""","""Ya""","""Ya""","""Pajak Pertambahan Nilai""",null
null,2024-02-05,"""TIKTOK""","""OL.2024.02.00304""","""MAKUKU""","""MAKUKU ADDCPM30""","""MAKUKU DRY CARE PANTS M30""","""ONLINE""",1.0,44299.0,0.0,12.5,0.0,null,44300.1,44299.0,43919.925,43919.925,380.175,380.175,5.7848e17,"""TK2FN-FEIZAL""","""INSTAN""","""Ya""","""Ya""","""Pajak Pertambahan Nilai""",null
null,2024-02-03,"""YOGYA SUBANG""","""GIT.2024.02.00025""","""OLIKE ACC""","""OLIKE CPH HC1 BLACK""","""OLIKE CAR PHONE HOLDER HC1 BLA…","""YOGYA SUBANG""",1.0,69520.0,8690.0,12.5,0.0,null,60830.22,60830.0,45999.998658,45999.998659,14830.221341,14830.221341,5.7848e17,"""CC-CHRISTIAN CHANDRA""","""03 februari 2024 GIT ACC PROMO""","""Ya""","""Ya""","""Pajak Pertambahan Nilai""",null
